## Import

In [ ]:
import os, sys, inspect, time

import numpy as np
import torch 
import matplotlib.pyplot as plt
torch.multiprocessing.set_sharing_strategy('file_system')

import discrepancy, visualization
from algorithms import ABC_algorithms, TPABC, SMCABC, SMC2ABC, SNLABC, SNL2ABC
import distributions 
import scipy.stats as stats

import utils_os, utils_math

%load_ext autoreload
%autoreload 2

## Problem Definition

In [ ]:
from problems.ABC_problems import ABC_Problem

class Neuronal_Problem(ABC_Problem):
    
    def __init__(self, data, N=100, n=100):
        
        assert N <= data['Y'].shape[0]
        assert data['Y'].ndim == 2
        assert data['X'].shape[0] == data['Y'].shape[0]
        
        self.N = N # number of posterior samples
        self.n = n # length of the data vector x = {x_1, ..., x_n} # makes sense to make it ~num_trials
#         self.d = 5 # dims of sufficient statistics? d=2K? This argument is just not used anywhere... great
        self.prior_args = np.array([[0,1]]) # these are bounds on theta (on X in our case: [0,1])
        
        self.all_thetas = data['X']
        self.sim_accuracy = 5 # number of digits after a decimal point for theta
        self.sim = {np.round(data['X'][i],self.sim_accuracy): data['Y'][i] for i in range(data['X'].shape[0])} #here we use all!
        self.K = 1 # number of thetas
        self.stat = 'raw' # raw means that sufficient statistics is unknown (I guess). y_obs = data_obs
        
        self.data_obs = data['Y'] #important that first dim=N & y_dim = product of these dims
        # y_obs is calculated from these data as y=statistics(data). 
        # note that y_obs is a argument of a Algorithm class, not Problem (¯\_(ツ)_/¯)
        
        self.is_batch_sampling_supported = False # (unfinished feature, so keep False for now) speed up rejection sampling
    
    def get_true_theta(self):
        pass # does not matter, as the result goes into 'statistics', where theta is currently not used

    def sample_from_prior(self, size=1):
        return np.random.choice(self.all_thetas,size=size,replace=True) # just 1 sample
    
    # original code samples only 1 theta in each simulation -> generates n x-es -> 
    # calculates statistics for them (1 vector for 1 theta) -> repeats ~1000 times sequentially (!)
    def simulator(self, theta):
        assert theta.size==1
        y = np.empty((self.n,self.data_obs.shape[1])) 
        for i in range(self.n): 
            t = np.round(theta[0] + (np.random.rand()-0.5)*0.01,self.sim_accuracy) # add jitter, to sample from the neighbouring locations
            if t in self.sim:
                y[i] = self.sim[t]
            else: # this part is used for newly-generated samples; let's take the Y=Y(closest X).
                discr = np.abs(self.all_thetas - t) # get distances
                y[i] = self.sim[np.round(self.all_thetas[np.argmin(discr)],self.sim_accuracy)] # take the closest
        return y # self.n x number of dimensions in data

    # B. correlation between latent
    def _ss_corr(self, Z):
        V = np.mat(Z).T * np.mat(Z) / Z.shape[0]
        (d,d) = V.shape
        upper_tri_elements = V[np.triu_indices(d, k=1)]
        stat = np.array(upper_tri_elements)
        return stat
    
    def statistics(self, data, theta=None):
        if self.stat == 'raw':
            # (correlation) as summary statistics (NO MARGINALS in these data)
            stat = self._ss_corr(data)
            return stat
        else:
            raise NotImplementedError('No ground truth statistics')

In [19]:
import pickle as pkl
with open(f'/home/nina/CopulaGP/plos_fig5_data/ST260_Day1_Dataset.pkl',"rb") as f:
    data = pkl.load(f)
    
Nvar = 10 # taking the first N variables here
data['Y'] = data['Y'][:,:Nvar] 
print(data['Y'].shape) # samples x neuronal/behavioral variables
    
problem = Neuronal_Problem(data)

DIR = 'results/Neuronal' 

(21471, 10)


In [20]:
### Sequential Neural Likelihood + 
hyperparams = ABC_algorithms.Hyperparams()
hyperparams.save_dir = DIR
hyperparams.device = 'cuda:0'
hyperparams.num_sim = 1000                        # number of simulations
hyperparams.L = 5                                # number of learning rounds
hyperparams.hidden_ratio = 0.1                   # dimensionality of S(x)
hyperparams.type = 'plain'                       # the network architecture of S(x), use CNN here
hyperparams.estimator = 'DV'                    # MI estimator; JSD or DC, see the paper
# 'DV' = proper MINE from Belghazi 2018
hyperparams.nde = 'MAF'                          # nde; MAF (D>1) or MDN (D=1) # looks like D here is in fact K

snl2_abc = SNL2ABC.SNL2_ABC(problem, discrepancy=discrepancy.eculidean_dist, hyperparams=hyperparams)


In [ ]:
snl2_abc.run()


iteration  0
# of cpus =  4
[ABC] sub-process start!
[sampling] finished sampling  10
[sampling] finished sampling  20
[ABC] sub-process start!
[sampling] finished sampling  30
[sampling] finished sampling  40
[ABC] sub-process start!
[ABC] sub-process start!

 > fitting encoder
summary statistic dim = 4 original dim = 45
architecture [45, 100, 100, 4]
validation size= 0.8
finished: t= 0 loss= 2.355128526687622e-05 loss val= 7.160007953643799e-06 time= 0.06008267402648926
finished: t= 50 loss= -4.793889820575714e-05 loss val= -3.099068999290466e-05 time= 0.06429839134216309
finished: t= 100 loss= -0.00011315383017063141 loss val= -6.365589797496796e-05 time= 0.06316351890563965
finished: t= 150 loss= -0.00022386759519577026 loss val= -0.0001295078545808792 time= 0.06337785720825195
finished: t= 200 loss= -0.00045620277523994446 loss val= -0.0003193523734807968 time= 0.06281161308288574
finished: t= 250 loss= -0.0010997336357831955 loss val= -0.0007021557539701462 time= 0.06459736824035

finished: t= 2050 loss= -0.5566120147705078 loss val= -0.48240596055984497 time= 0.10230898857116699
finished: t= 2100 loss= -0.5611029267311096 loss val= -0.48560208082199097 time= 0.10588335990905762
finished: t= 2150 loss= -0.5689806938171387 loss val= -0.4888806641101837 time= 0.10399627685546875
finished: t= 2200 loss= -0.5713651180267334 loss val= -0.5013905167579651 time= 0.10353422164916992
finished: t= 2250 loss= -0.5737749338150024 loss val= -0.49033012986183167 time= 0.1038355827331543
finished: t= 2300 loss= -0.5885589718818665 loss val= -0.4954584836959839 time= 0.1035609245300293
finished: t= 2350 loss= -0.5811280012130737 loss val= -0.5000746846199036 time= 0.1008749008178711
finished: t= 2400 loss= -0.5959227681159973 loss val= -0.4994288980960846 time= 0.10441875457763672
finished: t= 2450 loss= -0.5967461466789246 loss val= -0.5066975951194763 time= 0.10544919967651367
finished: t= 2500 loss= -0.5968956351280212 loss val= -0.5021742582321167 time= 0.10324311256408691


best val loss= -0.37690871953964233

 > fitting nde
all_stats.size() torch.Size([600, 4])
finished: t= 0 loss= 4.303411960601807 loss val= 4.137106895446777
best val loss= -8.844435691833496

 > fitting proposal
mu= [[0.72572182]]
cov= [[0.03691813]]


iteration  3
# of cpus =  4
[ABC] sub-process start!
[sampling] finished sampling  10
[sampling] finished sampling  20
[sampling] finished sampling  30
[ABC] sub-process start!
[sampling] finished sampling  40
[ABC] sub-process start!
[ABC] sub-process start!

 > fitting encoder
summary statistic dim = 4 original dim = 45
architecture [45, 100, 100, 4]
validation size= 0.8
finished: t= 0 loss= -2.4922192096710205e-06 loss val= -9.685754776000977e-06 time= 0.08285315831502278
finished: t= 50 loss= -0.0004916787147521973 loss val= -0.00044563040137290955 time= 0.07020028432210286
finished: t= 100 loss= -0.003825962543487549 loss val= -0.0036005377769470215 time= 0.06931233406066895
finished: t= 150 loss= -0.04923224449157715 loss val= -0.0

In [ ]:
# theta = snl2_abc.problem.sample_from_prior(size=20000)
# net = snl2_abc.nde_net
# y_obs, theta = snl2_abc.convert_stat(snl2_abc.whiten(snl2_abc.y_obs)), theta
# # y_obs = np.repeat(y_obs,400,axis=0)
# print(y_obs.shape)
# y_obs, theta = torch.tensor(y_obs).float(), torch.tensor(theta).float().view(1, -1)
# log_probs = net.log_probs(inputs=y_obs, cond_inputs=theta)
snl2_abc.problem.data_obs

In [ ]:
# let us check that the prior did not collapse 
theta = np.empty(1000)
for i in range(len(theta)): 
    theta[i] = snl2_abc.prior()
plt.xlim([0,1])
plt.hist(theta)

In [11]:
# get latents s(x)
snl2_abc.convert_stat(snl2_abc.problem.statistics(data['Y']))

array([[0.31910545, 0.31400834, 0.31528498, 0.31293586, 0.3094438 ,
        0.30839044, 0.30834512, 0.30658993, 0.30696861, 0.31580506,
        0.31325759, 0.31492224, 0.30785461, 0.30674789, 0.29722566,
        0.30548434, 0.29056185, 0.30904688, 0.31521884, 0.31955345,
        0.31123648, 0.29890438, 0.32111345, 0.28171838, 0.30982338,
        0.30561098, 0.30197579, 0.29698256, 0.30139333, 0.28665786,
        0.30832488, 0.30771774, 0.29709593, 0.30674394, 0.28172374,
        0.30755499, 0.2958246 , 0.31737984, 0.27935311, 0.29927007,
        0.30416235, 0.27669616, 0.29417504, 0.28358472, 0.27666595]])

In [ ]:
# calculate MI using all generated subsamples
all_stats = torch.tensor(np.vstack(snl2_abc.all_stats[0:snl2_abc.l+1])).float()
all_samples = torch.tensor(np.vstack(snl2_abc.all_samples[0:snl2_abc.l+1])).float()
print(all_samples.shape)
snl2_abc.vae_net.MI(all_stats,all_samples,n=100) # n here is the number of shuffles

In [ ]:
# calculate MI using the last generated subsamples
all_stats = torch.tensor(np.vstack(snl2_abc.all_stats[snl2_abc.l:snl2_abc.l+1])).float()
all_samples = torch.tensor(np.vstack(snl2_abc.all_samples[snl2_abc.l:snl2_abc.l+1])).float()
print(all_samples.shape)
snl2_abc.vae_net.MI(all_stats,all_samples,n=100) # n here is the number of shuffles

In [ ]:
# calculate MI using newly picked samples 
# (here: statistics of variable size, but pooled from the fixed neighbourhood)
new_stats = []
new_samples = []
for i in range(1000): # sample 1000 theta samples with replacement
    theta = snl2_abc.problem.sample_from_prior()
    mask = (data['X']>theta-1e-3) & ((data['X']<=theta+1e-3)) # we'll gather statistics from the neighbourhood of theta
    stats = snl2_abc.problem.statistics(data['Y'][mask]) # the number of samples is variable here, but the size of the neighbourhood is fixed
    samples = data['X'][mask].mean() # take mean theta from the neighbourhood (could as well just take theta)
    new_stats.append(stats)
    new_samples.append(samples) #theta 
new_stats = torch.tensor(new_stats).float().squeeze()
new_samples = torch.tensor(new_samples).float().reshape((-1,1))
print(new_stats.shape,new_samples.shape)
snl2_abc.vae_net.MI(new_stats,new_samples,n=100) # n here is the number of shuffles

# MINE for comparison

In [ ]:
from mine import train_MINE # load another implementation, the one I used for PLoS

In [ ]:
train_MINE(data['Y'][:,:Nvar], x=torch.tensor(data['X'][:]).float(), 
           H=1000, lr=0.01, batches=1, n_epoch=2000, device = torch.device("cuda:0"))